In [1]:
!pip install mesh_to_sdf tf_siren
import numpy as np
import tensorflow as tf
from mesh_to_sdf import get_surface_point_cloud
from mesh_to_sdf.utils import sample_uniform_points_in_unit_sphere
import trimesh
from tf_siren import SIRENModel
import re

     |████████████████████████████████| 1.2 MB 7.3 MB/s 
     |████████████████████████████████| 635 kB 49.4 MB/s 
     |████████████████████████████████| 890 kB 48.2 MB/s 
     |████████████████████████████████| 2.2 MB 70.4 MB/s 
  Created wheel for pyopengl: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745210 sha256=c3449c812379560a4c3f87f352fecbf876780e79ef851981da4cd7bbce5be459
  Stored in directory: /root/.cache/pip/wheels/c6/83/cb/af51a0c06c33d08537b941bbfc87469e8a3c68d05f77a6a212
Successfully built pyopengl
  Attempting uninstall: pyopengl
    Found existing installation: PyOpenGL 3.1.5
    Uninstalling PyOpenGL-3.1.5:
      Successfully uninstalled PyOpenGL-3.1.5


In [2]:
seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
def SDFFitting(filename, samples):
    surface_samples = int(np.floor(0.7 * samples))
    volume_samples = samples - surface_samples
    mesh = trimesh.load(filename)
    surface_point_cloud = get_surface_point_cloud(mesh, surface_point_method='sample')
    coords1, samples1 = surface_point_cloud.sample_sdf_near_surface(surface_samples, use_scans=False, sign_method='normal')
    coords2 = sample_uniform_points_in_unit_sphere(volume_samples)
    samples2 = surface_point_cloud.get_sdf_in_batches(coords2, use_depth_buffer=False)
    coords = np.concatenate([coords1,  coords2])
    samples = np.concatenate([samples1, samples2])
    return tf.data.Dataset.from_tensor_slices((coords, samples))

In [4]:
sdf = SDFFitting("bunny2.obj",256*256*4)

In [6]:
train_dataset = sdf.shuffle(10000).batch(65536).cache()
model = SIRENModel(
    units=16,   # Model enlarges with the square of this
    num_layers=2, # Model enlarges proportionally to this
    final_units=1, # 1 channel output - distance
    final_activation='linear',
    w0_initial=15., # Metaparameter
    w0=20.0,
)
_ = model(tf.zeros([1,3]))

num_steps = 1000 # 40000
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(0.005, decay_steps=num_steps, end_learning_rate=0.0005, power=2.0)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#loss = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)
#loss = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
loss = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)
model.compile(optimizer, loss=loss)
model.fit(train_dataset, epochs=num_steps, verbose=2)

Epoch 1/1000
4/4 - 1s - loss: 0.0239
Epoch 2/1000
4/4 - 0s - loss: 0.0230
Epoch 3/1000
4/4 - 0s - loss: 0.0222
Epoch 4/1000
4/4 - 0s - loss: 0.0216
Epoch 5/1000
4/4 - 0s - loss: 0.0212
Epoch 6/1000
4/4 - 0s - loss: 0.0213
Epoch 7/1000
4/4 - 0s - loss: 0.0202
Epoch 8/1000
4/4 - 0s - loss: 0.0191
Epoch 9/1000
4/4 - 0s - loss: 0.0180
Epoch 10/1000
4/4 - 0s - loss: 0.0172
Epoch 11/1000
4/4 - 0s - loss: 0.0168
Epoch 12/1000
4/4 - 0s - loss: 0.0162
Epoch 13/1000
4/4 - 0s - loss: 0.0149
Epoch 14/1000
4/4 - 0s - loss: 0.0135
Epoch 15/1000
4/4 - 0s - loss: 0.0145
Epoch 16/1000
4/4 - 0s - loss: 0.0123
Epoch 17/1000
4/4 - 0s - loss: 0.0110
Epoch 18/1000
4/4 - 0s - loss: 0.0138
Epoch 19/1000
4/4 - 0s - loss: 0.0102
Epoch 20/1000
4/4 - 0s - loss: 0.0090
Epoch 21/1000
4/4 - 0s - loss: 0.0076
Epoch 22/1000
4/4 - 0s - loss: 0.0075
Epoch 23/1000
4/4 - 0s - loss: 0.0074
Epoch 24/1000
4/4 - 0s - loss: 0.0062
Epoch 25/1000
4/4 - 0s - loss: 0.0063
Epoch 26/1000
4/4 - 0s - loss: 0.0058
Epoch 27/1000
4/4 - 0

In [8]:
def model_to_shadertoy(model):
    out = "float scene(vec3 p) {\n"
    out += "  if (length(p) > 1.) return length(p)-.8;\n"
    out += "  vec4 "
    vec4_defs = ["x=vec4(p,1)"]
    
    layers = len(model.weights) // 2
    assert(model.weights[0].shape[0]==3) # Input dimensionality of 3
    assert(model.weights[-1].shape==1) # Output dimensionality of 1
    
    def vec4(n):
        return 'vec4(' + ','.join(['{0:.2f}'.format(i) for i in n.flatten()]) + ')'

    def mat4(n, transpose=False):
        if transpose: n = np.transpose(n)
        return 'mat4(' + ','.join(['{0:.2f}'.format(i) for i in n.flatten()]) + ')'

    def vname(layer, chunk):
        return "f%d%d" % (layer, chunk)

    # First layer
    # Input vector includes 3 axes and a '1' term, to integrate bias.
    input_dim, output_dim = model.weights[0].shape
    assert(output_dim % 4 == 0)
    w0 = model.siren_layers.layers[0].w0
    for chunk in range(output_dim // 4):
        mat = np.concatenate([
            w0*model.weights[0][2, chunk*4:4+chunk*4],
            w0*model.weights[0][0, chunk*4:4+chunk*4],
            w0*model.weights[0][1, chunk*4:4+chunk*4],
            w0*model.weights[1][chunk*4:4+chunk*4]
        ]).reshape(4,4)
        vec4_defs.append(
            '{}=sin(x*{})'.format(
                vname(0,chunk),
                mat4(mat, transpose=True)
            )
        )
    
    # Hidden layers
    for layer in range(1, layers-1):
        w0 = model.siren_layers.layers[layer].w0
        input_dim, output_dim = model.weights[2*layer].shape
        assert(input_dim % 4 == 0)
        assert(output_dim % 4 == 0)
        for out_chunk in range(output_dim // 4):
            elements = []
            for in_chunk in range(input_dim // 4):
                elements.append(
                    mat4(w0*model.weights[2*layer][in_chunk*4:4+in_chunk*4, out_chunk*4:4+out_chunk*4].numpy(), transpose=False)
                    + '*'
                    + vname(layer-1, in_chunk)
                )
            elements.append(
                vec4(w0*model.weights[2*layer+1][out_chunk*4:4+out_chunk*4].numpy())
            )
            vec4_defs.append(
                '{}=sin({})'.format(
                    vname(layer, out_chunk),
                    '+'.join(elements)
                )
            )
        
    # Build into first statement
    out += ',\n    '.join(vec4_defs) + ";\n"

    # Output layer is separate return statement
    layer = layers - 1
    elements = []
    input_dim, output_dim = model.weights[2*layer].shape
    assert(input_dim % 4 == 0)
    assert(output_dim == 1)
    w0 = model.siren_layers.layers[-1].w0
    for in_chunk in range(input_dim // 4):
        elements.append(
            "dot({},{})".format(
                vec4(model.weights[2*layer][4*in_chunk:4+4*in_chunk].numpy()),
                vname(layer - 1, in_chunk)
            )
        )
    elements.append("{0:.2f}".format(model.weights[2*layer+1][0]))
    out += "  return " + "+".join(elements) + ";\n"
    out += "}\n"

    # Simplifying substitutions
    out = re.sub(r"(\d+\.\d*)0+\b", r"\1", out) # Remove trailing zeros eg. 1.0 => 1.
    out = re.sub(r"\b(\.\d+)0+\b", r"\1", out) # Remove trailing zeros eg. .60 => .6
    out = re.sub(r"\b0(\.\d+)\b", r"\1", out) # Remove leading zeros eg. 0.5 => .5
    out = re.sub(r"-\.0+\b", r".0", out) # Make all zeros positive eg. -.0 => .0
    out = re.sub(r"\+-", r"-", out) # Change +-1. into -1.

    return out

In [9]:
print(model_to_shadertoy(model))

float scene(vec3 p) {
  if (length(p) > 1.) return length(p)-.8;
  vec4 x=vec4(p,1),
    f00=sin(x*mat4(-1.31,-.26,-1.53,-1.31,3.05,4.56,-1.94,9.37,-2.57,1.89,2.41,8.36,-1.1,-2.77,1.75,.11)),
    f01=sin(x*mat4(3.83,.62,-2.31,.8,1.9,-3.72,-.13,1.97,3.25,.24,1.03,-5.17,-1.44,-.77,-1.64,-6.53)),
    f02=sin(x*mat4(3.93,5.07,-2.61,-2.99,-2.38,4.71,-.69,7.28,1.8,2.86,-.26,1.79,4.0,-1.8,-3.54,-5.46)),
    f03=sin(x*mat4(1.12,-.96,-3.65,.54,3.72,-1.41,.18,5.34,-3.8,-1.7,-4.07,3.16,-2.86,-.29,-3.11,-7.11)),
    f10=sin(mat4(2.23,-.11,1.49,.49,.96,-.58,.42,-.1,-1.08,.46,.1,.32,.83,.51,-.9,.73)*f00+mat4(-1.79,.57,.1,.66,.85,-.46,.13,-.65,-.62,1.27,-.06,.69,2.52,-.07,.82,.48)*f01+mat4(-.86,.46,-.23,.05,1.23,-.18,.05,.28,2.02,-.34,.54,.13,1.34,-.84,-.01,.22)*f02+mat4(1.94,-.29,.25,-.09,.17,.65,-.14,-.53,.43,.25,.15,.07,1.17,-.82,-.42,-.5)*f03+vec4(-9.24,8.67,.11,4.79)),
    f11=sin(mat4(.13,.01,-.37,-.63,.14,.13,1.2,.6,-.22,.32,-.53,-.34,.06,-.28,.57,.84)*f00+mat4(-.29,.23,-.56,-.52,.77,.36,-.28,